In [ ]:
from mpasanalysis import *
%matplotlib inline

In [ ]:
# flags
save_fig = False

In [ ]:
# get hostname
hostname = os.getenv('HOSTNAME')
print('Running on machine {}'.format(hostname))

In [ ]:
# set paths
if 'theta' in hostname:
    data_root = '/projects/ClimateEnergy_3/azamatm/E3SM_simulations/theta.20180906.branch_noCNT.A_WCYCL1950S_CMIP6_HR.ne120_oRRS18v3_ICG'
    fig_root = '/home/qingli/work/e3sm_res_cmp/figures/high_res'
    rst_file = data_root+'/run/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/run/mpaso.hist.am.timeSeriesStatsMonthly.0051-01-01.nc'
elif 'edison' in hostname:
    data_root = '/global/cscratch1/sd/tang30/ACME_simulations/edison.20181204.noCNT.A_WCYCL1950S_CMIP6_LRtunedHR.ne30_oECv3_ICG'
    fig_root = '/global/homes/q/qingli/work/e3sm_res_cmp/figures/low_res'
    rst_file = data_root+'/run/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/archive/ocn/hist/mpaso.hist.am.timeSeriesStatsMonthly.0041-01-01.nc'
else:
    raise EnvironmentError('This script should be executed on either edison or theta')
os.makedirs(fig_root, exist_ok=True)

In [ ]:
# load dataset
f_rst = Dataset(rst_file, 'r')
f_mon = Dataset(mon_file, 'r')

In [ ]:
# read grid information
lon = np.degrees(f_rst.variables['lonCell'][:])
lat = np.degrees(f_rst.variables['latCell'][:])
cellArea = f_rst.variables['areaCell'][:]

In [ ]:
# get variable list
var_keys = f_mon.variables.keys()
var_list = []
exclude_keywords = ['avgValue', 'minGlobalStats', 'maxGlobalStats', 'sumGlobalStats',
                    'rmsGlobalStats', 'avgGlobalStats', 'vertSum', 'area',
                    'vertNonLocalFlux',
                    'nonLocalSurfaceTracerFlux',
                    'activeTracerHorizontalAdvectionTendency',
                    'activeTracerVertMixTendency',
                    'activeTracerSurfaceFluxTendency',
                    'activeTracersSurfaceFluxRemoved',
                    'activeTracersSurfaceFluxRunoff',
                    'activeTracersSurfaceFlux',
                    'activeTracerVerticalAdvectionTendency',
                    'activeTracerNonLocalTendency',
                    'activeTracersTend',
                   ]
for var in var_keys:
    f_inc = True
    if 'timeMonthly_avg_' not in var:
        f_inc = False
        continue
    for exc in exclude_keywords:
        if exc in var:
            f_inc = False
            continue
    if f_inc:
        var_list.append(var[16:])
print(var_list)

In [ ]:
# plot figures
nCells = f_mon.dimensions['nCells'].size
for var_name in var_list:
    # load data
    ncvar = f_mon.variables['timeMonthly_avg_'+var_name]
    # check data
    ndim = len(ncvar.shape)
    if ndim == 2:
        dat = ncvar[0,:]
    elif ndim == 3:
        dat = ncvar[0,:,0]
    else:
        print('Skip \'{}\''.format(var_name))
        continue
    if dat.size != nCells:
        print('Skip \'{}\''.format(var_name))
        continue
    # create MPASOMap object
    units = ncvar.units
    mpaso_obj = MPASOMap(data=dat, lat=lat, lon=lon, cellarea=cellArea, name=var_name, units=units)
    # plot
    fig = plt.figure()
    fig.set_size_inches(6, 6)
    mpaso_obj.plot(region='LabSea')
    plt.show()
    plt.close(fig)
